In [7]:
from plotting_utils import *
import matplotlib.pyplot as plt
from cycler import cycler
plt.style.use('ggplot')

# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 8
#plt.rcParams['figure.titlesize'] = 15
#plt.rcParams['figure.titleweight'] = 'bold'

#plt.rc('lines', linewidth=4)
#plt.rc('axes', prop_cycle=(cycler(color=['r', 'g', 'b', 'y']) +
#                           cycler(linestyle=['-', '--', ':', '-.'])))
plt.rc('axes', prop_cycle=(cycler(color=['red', 'green', 'blue', 'yellow','cyan', 'magenta',\
                                         'black', 'orange', 'maroon', 'lime', 'aqua', \
                                         'indigo', 'darkviolet', 'dimgray', 'deeppink'])))


def plot2(ax, data, field, plot_term, params, smooth = None):
    '''data is already in filtered form
    data: {name:list of data_points} '''
    def last(a):
        return a[-1]

    colours = ['red', 'green', 'blue', 'yellow','cyan', 'magenta',\
                'black', 'orange', 'maroon', 'lime', 'aqua', \
                'indigo', 'darkviolet', 'dimgray', 'deeppink']

    all_plots = []
   # all_points = {}
    lookup = set()
    exclude_last_points = 1
    this_cmap = {}
    c_cpy = colours[:]
    c_taken = params["cmap"].values()
    for c in c_taken:
        if c in c_cpy:
            c_cpy.remove(c)
    for i,(k, v) in enumerate(data.items()):
        if plot_term in k:
            exp_id = k.split("_")
            ind = exp_id.index(plot_term)
            plot_term_value = exp_id[ind+1]
            if not plot_term_value in lookup:
                lookup.add(plot_term_value)
                label = params["aliasplotterm"] + ' = ' + plot_term_value
                if not smooth is None:
                    new_data = moving_average_filter(v[field], smooth)
                    exclude_last_points = smooth + 1
                else:
                    new_data = v[field][:]
                x = np.arange(len(new_data))
                
                if label in params["cmap"]:
                    c_hldr = params["cmap"][label]
                else:
                    c_hldr = c_cpy[0]
                    del c_cpy[0]
                this_cmap[label] = c_hldr
                #all_points[label] = (x, new_data, float(plot_term_value))
                handle = ax.plot(x[7:-exclude_last_points], new_data[7:-exclude_last_points], label = label, color = c_hldr)[0]
                all_plots.append((handle,label, float(plot_term_value)))


  #  all_points = {k: v for k, v in sorted(all_points.items(), key=lambda item: item[1][2], reverse = True)}

    handles, labels = ax.get_legend_handles_labels()
    label_id = [float(l.split(" = ")[-1]) for l in labels]
    all_plots = [tuple(i) for i in zip(handles, labels, label_id)]
   # all_plots.sort(key = last, reverse=True)


   # print(handles, labels)

    ax.set_xlabel(params["xlabel"], fontsize = params["xlabelsize"])  # Add an x-label to the axes.
    ax.set_ylabel(params["ylabel"], fontsize = params["ylabelsize"])  # Add a y-label to the axes.
    ax.set_title(params["title"], fontsize = params["titlesize"])  # Add a title to the axes.

  #  handles = [p[0] for p in all_plots]
   # lbls = [p[1] for p in all_plots]
    h,l,_ = zip(*all_plots)
    return h, l, this_cmap
    #ax.legend(h, l)  # Add a legend.

In [8]:
def plot4(ax, data,field, plts, params, smooth = None):
    ''' Plots the key-label pairs in plt where the key is a data item (single plot)'''
    colours = ['red', 'green', 'blue', 'yellow','cyan', 'magenta',\
                'black', 'orange', 'maroon', 'lime', 'aqua', \
                'indigo', 'darkviolet', 'dimgray', 'deeppink']
   # all_plots = []
    
    this_cmap = {}
    c_cpy = colours[:]
    c_taken = params["cmap"].values()
    for c in c_taken:
        if c in c_cpy:
            c_cpy.remove(c)
    
    for plt_key, plt_label in plts.items():
        assert plt_key in data
        #Data:
        d = data[plt_key]
        if not smooth is None:
            smooth_data = moving_average_filter(d[field], smooth)
            exclude_last_points = smooth + 1
        else:
            smooth_data = d[field]
            exclude_last_points = 1
        x = np.arange(len(smooth_data))
        #Colours:
        if plt_label in params["cmap"]:
            c_hldr = params["cmap"][plt_label]
        else:
            c_hldr = c_cpy[0]
            del c_cpy[0]
        this_cmap[plt_label] = c_hldr
        #Plot:
        handle = ax.plot(x[7:-exclude_last_points], smooth_data[7:-exclude_last_points], label = plt_label, color = c_hldr)[0]
      #  all_plots.append((handle,label, float(plot_term_value)))
        
        
    handles, labels = ax.get_legend_handles_labels()
    
    ax.set_xlabel(params["xlabel"], fontsize = params["xlabelsize"])  # Add an x-label to the axes.
    ax.set_ylabel(params["ylabel"], fontsize = params["ylabelsize"])  # Add a y-label to the axes.
    ax.set_title(params["title"], fontsize = params["titlesize"])  # Add a title to the axes.
    
    return handles, labels, this_cmap

In [54]:
from tabulate import tabulate
def get_mean_std(data, last_data, field, keys_alias_dict = None, print_ = True):
    '''Returns a table of mean std of data.
    keys_alias_dict can specify the keys in data to be
    plotted. The value item can specify an alias term, or
    be none to use the key as data label.'''
    csv_records = []
    #sep = ","
    #end_char = "\n"
    table = []
    hdrs = ["Item", "Mean", "Std"]
    if keys_alias_dict is None:
        for k,v in data.items():
            new_d = np.array(v[-last_data:])
            m = new_d.mean()
            std = new_d.std()
            hldr = [k,m,std]
            table.append(hldr)
            csv_records.append([m,std])
    else:
        for k, ak in keys_alias_dict.items():
            new_d = np.array(data[k][field][-last_data:])
            m = new_d.mean()
            std = new_d.std()
            if ak is None:
                ak = k
            hldr = [ak,m,std]
            table.append(hldr)
            csv_records.append(hldr)

    if print_:
        print(tabulate(table, headers = hdrs))
    return csv_records
        
            
    

In [10]:
source = '/home/james/Desktop/Gridworld/CENTRAL_TENSORBOARD/Ic3_1E'
all_data = get_event_data(source)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [11]:
plts = {}
labels = ["No Communication", "CommNet", "IC3Net"]
plts['IC3_disc_0.9_entropy_0.01_minibatch_500_commpasses_1_commzero_False_commNet_True_recurrent_False_envsize_5_nagents_4_objdensity_0.2_seed_1'] = "CommNet"
plts['IC3_disc_0.9_entropy_0.01_minibatch_500_commpasses_1_commzero_True_commNet_False_recurrent_False_envsize_5_nagents_4_objdensity_0.2_seed_1'] = "No Communication"

In [22]:
get_mean_std(all_data, 800,"total_agent_collisions", plts)

Item                 Mean      Std
----------------  -------  -------
CommNet           6.7015   2.71379
No Communication  6.86972  2.61161


In [53]:
a = 1.2489
print(np.round(a, 1))

1.2


In [26]:
def make_aliases(keys, plt_term):
    aliases = {}
    for k in keys:
        split = k.split("_")
        ind = split.index(plt_term)
        aliases[k] = split[ind+1]
    return aliases
        
        

In [64]:
def make_csv(table_dict, name):
    ''' table_dict contains {header: [row_label, mean, std]}'''
    sep = ","
    line_end = "\n"
    rows = {}
    header = "Architecture"
    for k,v in table_dict.items():
        header += sep + k
        for r in v:
            m = str(np.round(r[1], 2))
            std = str(np.round(r[2],1))
            if not r[0] in rows:
                rows[r[0]] = r[0]
            rows[r[0]] += sep + m + "$\pm$" + std
    csv_txt = header + line_end
    for v in rows.values():
        csv_txt += v + line_end
    f = open(make_exist(name), 'w')
    f.write(csv_txt)
    f.close()
    #print(csv_txt)
        
    
            

                
        
    

# Architectures 10x10
No data necessary, mlp learns nothing, cnn learns something but performs very poorly

# Arc 7x7

In [55]:
source = '/home/james/Desktop/Gridworld/CENTRAL_TENSORBOARD/3A'
all_data = get_event_data(source, remove_duplicates = False)

In [62]:
filter_terms = ["envsize_7"]
f_data = filter(all_data, filter_terms)
del f_data["1C_arc_primal5_disc_0.5_lambda_1.0_entropy_0.01_minibatch_512_rollouts_256_workers_4_kepochs_8_envsize_7_nagents_4_objdensity_0.2_seed_1_N1"]
del f_data["1C_arc_primal4_disc_0.5_lambda_1.0_entropy_0.01_minibatch_512_rollouts_256_workers_4_kepochs_8_envsize_7_nagents_4_objdensity_0.2_seed_1"]
del f_data["1C_arc_primal2_disc_0.5_lambda_1.0_entropy_0.01_minibatch_512_rollouts_256_workers_4_kepochs_8_envsize_7_nagents_4_objdensity_0.2_seed_1"]

alias = make_aliases(f_data.keys(), "arc")

table_data = {}

field = "total_steps"
print("\n\n" + field)
table_data[field] = get_mean_std(f_data, 100,field, alias, True)

field = "total_agent_collisions"
print(field)
table_data[field] = get_mean_std(f_data, 100,field, alias, True)


field = "total_obstacle_collisions"
print("\n\n" + field)
table_data[field] = get_mean_std(f_data, 100,field, alias, True)



    



total_steps
Item        Mean       Std
-------  -------  --------
primal2  61.9032  10.2153
mlp      66.6037   8.37039
primal7  58.9815  10.4188
primal3  71.3205   4.73592
primal5  70.122    6.85766
primal4  71.8518   4.6998
total_agent_collisions
Item         Mean      Std
-------  --------  -------
primal2   3.88781  2.8155
mlp       5.57317  3.54595
primal7   2.39571  1.67657
primal3  11.4707   4.69766
primal5  10.2668   6.40104
primal4  11.5333   6.65031


total_obstacle_collisions
Item        Mean      Std
-------  -------  -------
primal2  15.4325  7.31493
mlp      21.0845  8.3444
primal7  10.2814  3.9802
primal3  22.9875  6.63478
primal5  23.1933  8.69923
primal4  21.8732  7.27595


In [57]:
print(table_data)

{'total_agent_collisions': [['primal2', 3.8878095272183417, 2.8154972512641416], ['mlp', 5.573166670203209, 3.545953011083558], ['primal7', 2.3957142904400826, 1.676569414738153], ['primal3', 11.47066665649414, 4.697659054612068], ['primal5', 10.266833329200745, 6.401043069260606], ['primal4', 11.533333335518837, 6.650305361448767]], 'total_obstacle_collisions': [['primal2', 15.43250001192093, 7.314934552610354], ['mlp', 21.084499979019164, 8.344403085265277], ['primal7', 10.281357126235962, 3.980200679337418], ['primal3', 22.9875, 6.634782255346958], ['primal5', 23.193333344459532, 8.699227623311653], ['primal4', 21.873166646957397, 7.275954656465044]], 'total_steps': [['primal2', 61.9032141494751, 10.215300310405178], ['mlp', 66.60366672515869, 8.370390787808647], ['primal7', 58.9815238571167, 10.418806461890565], ['primal3', 71.3204999923706, 4.735923450200105], ['primal5', 70.12199996948242, 6.857663165583753], ['primal4', 71.85183326721192, 4.699797215766051]]}


In [66]:
where = "tables/Arc7x7.csv"
make_csv(table_data, where)

# Arc 5X5

In [68]:
filter_terms = ["envsize_5"]
f_data = filter(all_data, filter_terms)

alias = make_aliases(f_data.keys(), "arc")
table_data = {}

field = "total_agent_collisions"
print(field)
table_data[field] = get_mean_std(f_data, 100,field, alias,True)


field = "total_obstacle_collisions"
print("\n\n" + field)
table_data[field] = get_mean_std(f_data, 100,field, alias,True)


field = "total_steps"
print("\n\n" + field)
table_data[field] = get_mean_std(f_data, 100,field, alias, True)

total_agent_collisions
Item        Mean      Std
-------  -------  -------
primal2  4.38939  1.88407
primal6  1.85667  1.30216
mlp      2.60375  1.7318


total_obstacle_collisions
Item        Mean      Std
-------  -------  -------
primal2  7.98942  2.45689
primal6  4.08684  1.55103
mlp      7.29095  3.00721


total_steps
Item        Mean      Std
-------  -------  -------
primal2  31.7415  3.83791
primal6  25.4331  4.03124
mlp      29.0913  4.05137


In [69]:
where = "tables/Arc5x5.csv"
make_csv(table_data, where)